In [8]:
import os
import json
import base64
from requests import post
from dotenv import load_dotenv


In [4]:
load_dotenv()

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [6]:
def get_token():
    auth_string = client_id + ':' + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')

    url = 'https://accounts.spotify.com/api/token'
    headers= {
        'Authorization': 'Basic ' + auth_base64,
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data = {'grant_type': 'client_credentials'}

    result = post(url, headers=headers, data=data)

    json_result = json.loads(result.content)
    token = json_result['access_token']
    return token


In [9]:
get_token()

'BQA2YNlxGB8h9USAGatC1K72qd-PElGS_JFbq1aCrg3xDjdoutXze_ZQWcqF6dhLtxDd9vOXDITVLa_w6TOJnoHwIxry6Ntlzw1PyWWF4U3sFdd70uo'